In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline
import sys
import os
import numpy as np
import pandas as pd
import scipy.sparse
from scipy.sparse import csc_matrix
from scipy.sparse import lil_matrix

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Circle
import seaborn as sns; 
from IPython.display import HTML

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn import svm
from sklearn.metrics import r2_score

In [ ]:
#config parser
import configparser

sys.path.insert(0, '../Common/')
from AISDataManager import AISDataManager
import Constants as c
import HMUtils as hMUtil
import TimeUtils as timeUtils
import GeoCompute as gC

#MyConfig.INI stores all the run time constants
config = configparser.ConfigParser()
config.read('../MyConfig.INI')

from joblib import Parallel, delayed
import multiprocessing
aISDM = AISDataManager()
numCores = multiprocessing.cpu_count()

In [ ]:
lonMin = (float)(config['TP_SEC_ORDER']['LON_MIN'])
lonMax = (float)(config['TP_SEC_ORDER']['LON_MAX'])

latMin = (float)(config['TP_SEC_ORDER']['LAT_MIN'])
latMax = (float)(config['TP_SEC_ORDER']['LAT_MAX'])

print(lonMin,latMin)
print(lonMax,latMax)

increStep = (float)(config['TP_SEC_ORDER']['INCR_STEP'])
incrRes = (int)(config['TP_SEC_ORDER']['INCR_RES'])

trueDir = "../Data/M120_50_M119_00_33_90_34_44/Display/TrueTraj/"
probDir = "../Data/M120_50_M119_00_33_90_34_44/Display/ProbTraj/"
trueProbDir = "../Data/M120_50_M119_00_33_90_34_44/Display/TrueProbTraj/"
boundaryPad = 0.4

In [ ]:
trueImage = trueDir + '0' + '.png'    
vesselCurrImg = plt.imread(trueImage)
print(vesselCurrImg.shape)

In [ ]:
fig = plt.figure(figsize=(12,8))

ax1 = fig.add_axes([0.1, 0.1, 0.38, 0.70])
ax2 = fig.add_axes([0.56, 0.1, 0.38, 0.70])
ax3 = fig.add_axes([0.1, 0.72, 0.94, 0.10])

minuteCounter = 0
def animate(i):
    global minuteCounter 
    trueImage = trueDir + str(i) + '.png'
    probImage = probDir + str(i) + '.png'
    
    vesselCurrImg = plt.imread(trueImage)
    vesselCurrPosPredImg = plt.imread(probImage)
    ax1.imshow(vesselCurrImg[250:480,350:720],extent=[lonMin-boundaryPad \
                                                          ,lonMax+boundaryPad \
                                                          ,latMin-boundaryPad \
                                                          ,latMax+boundaryPad \
                                                         ])



    ax2.imshow(vesselCurrPosPredImg[250:480,350:720],extent=[lonMin-boundaryPad \
                                                              ,lonMax+boundaryPad \
                                                              ,latMin-boundaryPad \
                                                              ,latMax+boundaryPad \
                                                             ])
    

    ax1.set_xticks([lonMin-boundaryPad,lonMax+boundaryPad])
    ax1.set_yticks([latMin-boundaryPad,latMax+boundaryPad])
    
    ax2.set_xticks([lonMin-boundaryPad,lonMax+boundaryPad])
    ax2.set_yticks([latMin-boundaryPad,latMax+boundaryPad])
    
    ax1.set_xlabel('Longitude')
    ax1.set_ylabel('Lattitude')
    titleStr = 'Vessel Position %d'%((i))
    ax1.set_title(titleStr)
    
    ax2.set_xlabel('Longitude')
    ax2.set_ylabel('Lattitude')
    ax2.set_title('Transition Probability Projection')
    
    ax3.clear()
    ax3.get_xaxis().set_visible(False)
    ax3.get_yaxis().set_visible(False)
    textToDisp = "%03d MINUTES..."%(minuteCounter)
    minuteCounter = minuteCounter + 5
    ax3.text(0.30, 0.1, textToDisp, fontsize=36)
    ax3.axis('off')
    
anim = animation.FuncAnimation(fig, animate,frames=65, interval=500)

In [ ]:
fig = plt.figure(figsize=(10,8))

ax1 = fig.add_axes([0.1, 0.1, 0.88, 0.58])
ax3 = fig.add_axes([0.1, 0.72, 0.94, 0.10])

# minuteCounter = 0
def animate(i):
#     global minuteCounter 
    trueImage = trueProbDir + str(i) + '.png'
    
    vesselCurrImg = plt.imread(trueImage)
    ax1.imshow(vesselCurrImg[250:480,350:720],extent=[lonMin-boundaryPad \
                                                          ,lonMax+boundaryPad \
                                                          ,latMin-boundaryPad \
                                                          ,latMax+boundaryPad \
                                                         ])

    

    ax1.set_xticks([lonMin-boundaryPad,lonMax+boundaryPad])
    ax1.set_yticks([latMin-boundaryPad,latMax+boundaryPad])
        
    ax1.set_xlabel('Longitude')
    ax1.set_ylabel('Lattitude')
    titleStr = 'Vessel Position Prediction'
    ax1.set_title(titleStr)
    
    ax3.clear()
    ax3.get_xaxis().set_visible(False)
    ax3.get_yaxis().set_visible(False)
    minuteCounter = (i*5) + 30
    textToDisp = "%03d MINUTES"%(minuteCounter)
    ax3.text(0.30, 0.1, textToDisp, fontsize=36)
    ax3.axis('off')
    
anim = animation.FuncAnimation(fig, animate,frames=58, interval=500)

In [ ]:
HTML(anim.to_html5_video())

In [ ]:
Writer = animation.writers['ffmpeg']
writer = Writer(fps=2, metadata=dict(artist='Me'), bitrate=1800)

In [ ]:
anim.save('SixVesselVisualColor30WD.mp4', writer=writer)